In [ ]:
import torch as th

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
def make_policy_obs_action(seq_len, ntoken, d_out, diff_ele, device = 'cuda'):
    o1 = th.zeros([1, seq_len, ntoken], dtype=th.float, device=device)
    o2 = th.zeros_like(o1)
    o2[0, diff_ele] = 1

    a1 = th.zeros(1, seq_len, d_out, dtype=th.float, device=device)
    a2 = th.zeros_like(a1)
    a2[:, diff_ele:] = 1
    o = th.cat((o1, o2), dim=0)
    a = th.cat((a1, a2), dim=0)
    
    return o, a


In [ ]:
observations, actions = make_policy_obs_action(3, 2, 1, 1, 'cuda')

In [ ]:
from ActiveCritic.utils.pytorch_utils import make_partially_observed_seq
import numpy as np
import gym
import torch as th
seq_len = 3
d_out = 2
d_in = 3
acts_array_low = [0]*d_out
acts_array_high = [1]*d_out

obs = th.ones([2, 1, d_in], dtype=th.float, device='cuda')

In [ ]:
action_space = gym.spaces.box.Box(
    np.array(acts_array_low), np.array(acts_array_high), (d_out,), float)

pos = make_partially_observed_seq(obs=obs, acts=None, seq_len=seq_len, act_space=action_space)
expected_shape = list(obs.shape)
expected_shape[1] = seq_len
expected_shape[2] += action_space.shape[0]
assert list(pos.shape) == expected_shape, f'Output shape is not as expected. Expected: {expected_shape}, but got {pos.shape}'
assert th.all(pos[:,:,obs.shape[-1]:] == 0), 'No action input, but some action fields are not zero.'
assert th.equal(pos[:,:obs.shape[1], :obs.shape[2]], obs), 'Observation not correctly inserted into sequence.'

In [ ]:
current_obs_len = 2
obs = th.ones([2, current_obs_len, d_in], dtype=th.float, device='cuda')
acts = 2*th.ones([2, current_obs_len-1, d_out], dtype=th.float, device='cuda')
pos = make_partially_observed_seq(obs=obs, acts=acts, seq_len=seq_len, act_space=action_space)

In [ ]:
expected_shape = list(obs.shape)
expected_shape[1] = seq_len
expected_shape[2] += action_space.shape[0]
assert list(pos.shape) == expected_shape, f'Output shape is not as expected. Expected: {expected_shape}, but got {pos.shape}'
assert th.all(pos[:,acts.shape[1],obs.shape[-1]:] == 0), 'Action that werent input are displayed.'
assert th.equal(pos[:,:obs.shape[1], :obs.shape[2]], obs), 'Observation not correctly inserted into sequence.'
assert th.equal(pos[:,:acts.shape[1], obs.shape[2]:], acts), 'Not all actions have been inserted correctly.'

In [ ]:
import gym
from ActiveCritic.utils.gym_utils import make_policy_dict
from ActiveCritic.metaworld.metaworld.envs import ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE
from gym.wrappers import TimeLimit
from stable_baselines3.common.vec_env import DummyVecEnv
from imitation.data.wrappers import RolloutInfoWrapper
from ActiveCritic.utils.gym_utils import new_epoch_reach
import numpy as np
import torch as th
policy_dict = make_policy_dict()

env_tag = 'reach'
max_episode_steps = 5
re = ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE[policy_dict[env_tag][1]]()
re._freeze_rand_vec = False
timelimit = TimeLimit(env=re, max_episode_steps=max_episode_steps)
dv1 = DummyVecEnv([lambda: RolloutInfoWrapper(timelimit)])

In [ ]:
from ActiveCritic.utils.gym_utils import DummyExtractor
de = DummyExtractor()

obs1 = de.forward(dv1.reset())

In [ ]:
new_epoch_reach(obs1, obs1)

In [ ]:
obs2_np, _,_,_ = dv1.step(np.array([[0,1,0,0]]))

In [1]:
from active_critic.utils.gym_utils import make_dummy_vec_env, sample_expert_transitions, new_epoch_reach, DummyExtractor, parse_sampled_transitions, make_vec_env, parse_sampled_transitions_legacy
import torch as th
import numpy as np
seq_len = 100
episodes = 3
name = 'reach'

/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [2]:
env, exp = make_vec_env(env_id='reach', num_cpu=1, seq_len=100)

/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [3]:
transitions = sample_expert_transitions(policy=exp.predict, env=env, episodes=episodes)

Sampling transitions. 3


/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/metaworld/policies/policy.py:41: UserWarning: Constant(s) may be too high. Environments clip response to [-1, 1]
  warnings.warn('Constant(s) may be too high. Environments clip response to [-1, 1]')


In [4]:
actions, observations, rewards = parse_sampled_transitions(transitions=transitions, seq_len=seq_len, extractor=DummyExtractor())


In [6]:
epochs = [0]
for i, step in enumerate(transitions):
    if step['dones']:
        epochs.append(i)

In [11]:
epochs = th.tensor(epochs)
episodes_len = th.diff(epochs)
for i, reward in enumerate(rewards):
    assert th.all(reward[int(episodes_len[i]+1):] == -1)

In [13]:
episodes_len

tensor([52, 48, 42])

In [ ]:
actions_leg, observations_leg, rewards_leg = parse_sampled_transitions_legacy(transitions=transitions, new_epoch=new_epoch_reach, seq_len=seq_len, extractor=DummyExtractor())

In [ ]:
assert th.equal(actions, actions_leg)
assert th.equal(observations, observations_leg)
assert th.equal(rewards, rewards_leg)

In [ ]:
rewards[1][88] 

In [ ]:
th.tensor(np.array(transitions[127]['infos']['unscaled_reward'])) == 10

In [ ]:
assert list(actions.shape) == [episodes, seq_len, env.action_space.shape[0]]
assert list(observations.shape) == [episodes, seq_len, env.observation_space.shape[0]]
assert list(rewards.shape) == [episodes, seq_len]
assert th.all(rewards[:,-1] == 1)

In [ ]:
from active_critic.utils.gym_utils import fill_arrays
import numpy as np

In [ ]:
seq_len = 3
a = np.ones([3,2])
b = np.ones([2,2])
c = [a,b]
d = fill_arrays(np.array(c), seq_len=3)

In [ ]:
assert np.array(c).dtype == object
assert d.dtype is not object